# Relaciones

In [22]:
#conexiones a aura
from neo4j import GraphDatabase
uri = "neo4j+ssc://5fb4d30f.databases.neo4j.io"
auth= ("neo4j", "8zCnpPpgQmkE8vEsazT7nPT8y4DWHvZ9dq0aPaD5qVc")


In [19]:
with GraphDatabase.driver(URI, auth=AUTH) as driver:
    driver.verify_connectivity()

In [23]:
def crear_relaciones(uri, auth, relationship_type, properties_keys):

    empty_properties = {key: "" if isinstance(key, str) else 0 for key in properties_keys} #cre

    query = f"""
    MATCH ()-[r:{relationship_type}]->()
    SET r += $empty_properties
    RETURN COUNT(r) AS total_updated
    """

    with GraphDatabase.driver(uri, auth=auth) as driver:
        with driver.session() as session:
            result = session.run(query, empty_properties=empty_properties)
            return result.single()["total_updated"]

In [24]:
# Propiedades para ACTED_IN
properties_keys = ["Role", "ScreenTime", "CharacterImportance"]
relacion = "ACTED_IN"
total = crear_relaciones(uri, auth, relacion, properties_keys)


Se actualizaron {total} relaciones {relacion} con nuevas propiedades vacías


In [25]:
print(f"Se actualizaron {total} relaciones {relacion} con nuevas propiedades vacías")

Se actualizaron 62841 relaciones ACTED_IN con nuevas propiedades vacías


In [26]:
#propiedades para DIRECTED
properties_keys = ["Year", "AwardNominated", "FilmFestival"]
relacion = "DIRECTED"
total = crear_relaciones(uri, auth, relacion, properties_keys)
print(f"Se actualizaron {total} relaciones {relacion} con nuevas propiedades vacías")

Se actualizaron 4669 relaciones DIRECTED con nuevas propiedades vacías


In [27]:
#Prpiedades para directed by
properties_keys = ["Year", "AwardNominated", "FilmFestival"]
relacion = "DIRECTED_BY"
total = crear_relaciones(uri, auth, relacion, properties_keys)
print(f"Se actualizaron {total} relaciones {relacion} con nuevas propiedades vacías")

Se actualizaron 4669 relaciones DIRECTED_BY con nuevas propiedades vacías


In [29]:
#Prpiedades para FILMED_IN
properties_keys = ["City", "Days", "Year"]
relacion = "FILMED_IN"
total = crear_relaciones(uri, auth, relacion, properties_keys)
print(f"Se actualizaron {total} relaciones {relacion} con nuevas propiedades vacías")

Se actualizaron 49350 relaciones FILMED_IN con nuevas propiedades vacías


In [30]:
#Prpiedades para IS FRIENDS WITH
properties_keys = ["Since", "CommonInterests", "FrequenciaInteraccion"]
relacion = "IS_FRIENDS_WITH"
total = crear_relaciones(uri, auth, relacion, properties_keys)
print(f"Se actualizaron {total} relaciones {relacion} con nuevas propiedades vacías")

Se actualizaron 2026 relaciones IS_FRIENDS_WITH con nuevas propiedades vacías


In [31]:
#Prpiedades para IS TYPE
properties_keys = ["CommunityRating", "ReviewText", "MatchAccuracy"]
relacion = "IS_TYPE"
total = crear_relaciones(uri, auth, relacion, properties_keys)
print(f"Se actualizaron {total} relaciones {relacion} con nuevas propiedades vacías")

Se actualizaron 91015 relaciones IS_TYPE con nuevas propiedades vacías


In [32]:
#Prpiedades para OPERATS IN
properties_keys = ["Marketshare", "EmployeeCount", "YearsEstablished"]
relacion = "OPERATES_IN"
total = crear_relaciones(uri, auth, relacion, properties_keys)
print(f"Se actualizaron {total} relaciones {relacion} con nuevas propiedades vacías")

Se actualizaron 34853 relaciones OPERATES_IN con nuevas propiedades vacías


In [33]:
#Prpiedades para PRODUCED BY
properties_keys = ["Budget", "BoxOffice", "Reception"]
relacion = "PRODUCED_BY"
total = crear_relaciones(uri, auth, relacion, properties_keys)
print(f"Se actualizaron {total} relaciones {relacion} con nuevas propiedades vacías")

Se actualizaron 4 relaciones PRODUCED_BY con nuevas propiedades vacías


In [34]:
#Prpiedades para RATED
properties_keys = ["Ratingvalue", "RatingDate", "ReviewText"]
relacion = "RATED"
total = crear_relaciones(uri, auth, relacion, properties_keys)
print(f"Se actualizaron {total} relaciones {relacion} con nuevas propiedades vacías")

Se actualizaron 2514 relaciones RATED con nuevas propiedades vacías


In [35]:
#Prpiedades para RECOMMENDED_BY
properties_keys = ["Ratingvalue", "RatingDate", "ReviewText"]
relacion = "WAS_RECOMMENDED_BY"
total = crear_relaciones(uri, auth, relacion, properties_keys)
print(f"Se actualizaron {total} relaciones {relacion} con nuevas propiedades vacías")

Se actualizaron 2514 relaciones WAS_RECOMMENDED_BY con nuevas propiedades vacías


## Recomendador

In [39]:
import datetime
def menu():
    print("BIENVENIDO AL RECOMENDADOR DE PELICULAS")
    print("Ingresa tus gustos y preferencias a continuación:")


def validar_fecha(fecha_texto):
    try:
        datetime.datetime.strptime(fecha_texto, "%Y-%m-%d")
        return True
    except ValueError:
        return False

def procesar_entrada_lista(entrada, capitalizar=False):
    lista = entrada.split(',')
    if capitalizar:
        # Capitalizar la primera letra de cada palabra en cada elemento de la lista.
        return [item.strip().title() for item in lista]
    else:
        return [item.strip() for item in lista]

def variables_recomendacion():
    genero = procesar_entrada_lista(input("Ingresa los géneros que te gustan, separados por comas: "), True)
    
    while True:
        released = input("Ingresa la fecha de estreno que prefieres (formato YYYY-MM-DD): ").strip()
        if validar_fecha(released):
            break
        print("Formato de fecha incorrecto, por favor intenta de nuevo.")
    
    while True:
        duracion = input("Ingresa la duración mínima que prefieres para la película (en minutos): ").strip()
        if duracion.isdigit():
            duracion = int(duracion)
            break
        print("Por favor ingresa un número válido.")
    
    keywords = procesar_entrada_lista(input("Ingresa palabras clave relacionadas con la película, separadas por comas: "))
    studio = input("Ingresa el estudio de cine que prefieres: ").strip().title()
    director = input("Ingresa el nombre del director que te gustaría ver: ").strip().title()
    actors = procesar_entrada_lista(input("Ingresa los nombres de los actores que te gustan, separados por comas: "), True)

    return genero, released, duracion, keywords, studio, director, actors


#############################################

def neo4j_connect(uri, auth):
    driver = GraphDatabase.driver(uri, auth=auth)
    return driver

def get_movie_recommendations(driver, genero, released, duracion, keywords, studio, director, actors):
    # Ajustar el rango de fecha
    year = int(released.split("-")[0])
    min_year = year - 5
    max_year = year + 5
    
    # Ajustar el rango de duración
    min_runtime = duracion - 30
    max_runtime = duracion + 30
    
    query = """
    MATCH (m:Movie)<-[:ACTED_IN]-(a:Actor), (m)-[:IS_TYPE]->(g:Genre), (m)<-[:DIRECTED_BY]-(d:Director), (m)<-[:PRODUCED_BY]-(c:Company)
    WHERE 
        g.Name IN $genres AND
        m.Released >= $minYear AND m.Released <= $maxYear AND
        m.runtime >= $minRuntime AND m.runtime <= $maxRuntime AND
        any(keyword IN m.keywords WHERE keyword IN $keywords) AND
        c.Name = $studio AND
        d.name = $director AND
        any(actor IN collect(a.Name) WHERE actor IN $actors)
    RETURN m.title AS Title, m.Released AS Released, m.runtime AS Runtime
    LIMIT 10
    """
    
    with driver.session() as session:
        result = session.run(query, {
            'genres': genero,
            'minYear': min_year,
            'maxYear': max_year,
            'minRuntime': min_runtime,
            'maxRuntime': max_runtime,
            'keywords': keywords,
            'studio': studio,
            'director': director,
            'actors': actors
        })
        return list(result)

# Conexión a Neo4j

driver = neo4j_connect(uri, auth)

################### MAIN

menu()

# Recoger inputs del usuario
inputs = variables_recomendacion()
genero, released, duracion, keywords, studio, director, actors = inputs

# Obtener recomendaciones
recommendations = get_movie_recommendations(driver, *inputs)

#Imprimir recomendaciones
for movie in recommendations:
    print(movie['Title'], movie['Released'], movie['Runtime'])
############################################



BIENVENIDO AL RECOMENDADOR DE PELICULAS
Ingresa tus gustos y preferencias a continuación:


Ingresa los géneros que te gustan, separados por comas:  Comedy
Ingresa la fecha de estreno que prefieres (formato YYYY-MM-DD):  1995-12-10
Ingresa la duración mínima que prefieres para la película (en minutos):  90
Ingresa palabras clave relacionadas con la película, separadas por comas:  friends
Ingresa el estudio de cine que prefieres:  Pixar
Ingresa el nombre del director que te gustaría ver:  
Ingresa los nombres de los actores que te gustan, separados por comas:  


CypherSyntaxError: {code: Neo.ClientError.Statement.SyntaxError} {message: Invalid use of aggregating function collect(...) in this context (line 10, column 22 (offset: 454))
"        any(actor IN collect(a.Name) WHERE actor IN $actors)"
                      ^}